# SunGlass Hut Inventory Management - Solution

This notebook provides a complete solution for analyzing SunGlass Hut's inventory management strategy, including:
1. Probability Analysis
2. Service Level Optimization
3. Two-Stage System Design
4. Business Recommendations

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from utils.testing.inventory_tests import (
    check_service_level,
    check_z_score,
    check_two_stage_system,
    verify_inventory_levels
)

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')

## Part 1: Basic Probability Analysis

First, let's analyze the current inventory strategy.

In [ ]:
# Define parameters
mean = 75
std_dev = 25
stock_level = 90

# Calculate z-score
z_score = (stock_level - mean) / std_dev

# Calculate probabilities
prob_meeting_demand = stats.norm.cdf(z_score)
prob_stockout = 1 - prob_meeting_demand

print("Current Inventory Strategy Analysis:")
print("-" * 50)
print(f"Z-score for k=90: {z_score:.4f}")
print(f"Probability of Meeting Demand: {prob_meeting_demand:.4f} ({prob_meeting_demand*100:.1f}%)")
print(f"Probability of Stockout: {prob_stockout:.4f} ({prob_stockout*100:.1f}%)")

# Verify calculations
if check_z_score({'z_score': z_score}):
    print("\n✓ Z-score correctly calculated")
else:
    print("\n✗ Check your z-score calculation")

# Visualize the probability distribution
x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
y = stats.norm.pdf(x, mean, std_dev)

plt.figure(figsize=(12, 6))
plt.plot(x, y, 'b-', lw=2, label='Demand Distribution')
plt.fill_between(x[x <= stock_level], y[x <= stock_level],
                 color='g', alpha=0.3, label='Meeting Demand')
plt.fill_between(x[x > stock_level], y[x > stock_level],
                 color='r', alpha=0.3, label='Stockout')
plt.axvline(stock_level, color='k', linestyle='--', label='Stock Level (k=90)')

plt.title('Demand Distribution and Stock Level')
plt.xlabel('Monthly Demand (units)')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 2: Service Level Analysis

Now let's determine inventory levels needed for different service level targets.

In [ ]:
# Define service levels
service_levels = [0.90, 0.95]

# Calculate required inventory levels
z_values = stats.norm.ppf(service_levels)
inventory_levels = mean + z_values * std_dev

print("Service Level Analysis:")
print("-" * 50)
for sl, z, inv in zip(service_levels, z_values, inventory_levels):
    print(f"Service Level: {sl*100}%")
    print(f"Z-value: {z:.4f}")
    print(f"Required Inventory: {inv:.1f} units")
    print()

# Verify calculations
if check_service_level({'service_level': 0.95, 'inventory': inventory_levels[1]}):
    print("✓ Service level calculations correct")
else:
    print("✗ Check your service level calculations")

# Visualize service levels
plt.figure(figsize=(12, 6))
plt.plot(x, y, 'b-', lw=2, label='Demand Distribution')

colors = ['g', 'r']
for sl, inv, color in zip(service_levels, inventory_levels, colors):
    plt.axvline(inv, color=color, linestyle='--',
                label=f'{sl*100}% Service Level ({inv:.1f} units)')

plt.title('Service Level Requirements')
plt.xlabel('Monthly Demand (units)')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 3: Two-Stage System Design

Let's analyze a two-stage inventory system with a central warehouse.

In [ ]:
# Define two-stage system parameters
n_stores = 150
target_service_level = 0.95
lead_time_days = 3
review_period_days = 7

# Calculate aggregate demand parameters
daily_mean = mean / 30  # Convert monthly to daily
daily_std = std_dev / np.sqrt(30)  # Assuming independent daily demand

# Calculate parameters for review period + lead time
cycle_days = review_period_days + lead_time_days
cycle_mean = daily_mean * cycle_days
cycle_std = daily_std * np.sqrt(cycle_days)

# Calculate required safety stock
z_service = stats.norm.ppf(target_service_level)
safety_stock = z_service * cycle_std

# Calculate reorder point and order-up-to level
reorder_point = cycle_mean + safety_stock
order_up_to = reorder_point + daily_mean * review_period_days

print("Two-Stage System Design:")
print("-" * 50)
print("Store Level:")
print(f"Daily Mean Demand: {daily_mean:.1f} units")
print(f"Daily Std Dev: {daily_std:.1f} units")
print(f"Cycle Mean Demand: {cycle_mean:.1f} units")
print(f"Cycle Std Dev: {cycle_std:.1f} units")
print(f"Safety Stock: {safety_stock:.1f} units")
print(f"Reorder Point: {reorder_point:.1f} units")
print(f"Order-up-to Level: {order_up_to:.1f} units")

# Verify calculations
system_params = {
    'safety_stock': safety_stock,
    'reorder_point': reorder_point,
    'order_up_to': order_up_to
}

if check_two_stage_system(system_params):
    print("\n✓ Two-stage system correctly designed")
else:
    print("\n✗ Check your two-stage system calculations")

## Part 4: Business Recommendations

Let's analyze the business implications and make recommendations.

In [ ]:
# Calculate key metrics
current_service_level = prob_meeting_demand
current_total_inventory = stock_level * n_stores
proposed_store_inventory = order_up_to
proposed_total_inventory = proposed_store_inventory * n_stores

# Calculate stockout costs
unit_cost = 150  # Example cost per unit
margin = 0.4  # 40% profit margin
lost_sale_cost = unit_cost * margin
monthly_stockouts = prob_stockout * mean * n_stores
monthly_lost_revenue = monthly_stockouts * lost_sale_cost

print("Business Impact Analysis:")
print("-" * 50)
print("Current System:")
print(f"Service Level: {current_service_level*100:.1f}%")
print(f"Total Inventory: {current_total_inventory:,.0f} units")
print(f"Expected Monthly Stockouts: {monthly_stockouts:.0f} units")
print(f"Monthly Lost Revenue: ${monthly_lost_revenue:,.2f}")
print()
print("Proposed System:")
print(f"Target Service Level: {target_service_level*100:.1f}%")
print(f"Store Inventory Level: {proposed_store_inventory:.1f} units")
print(f"Total System Inventory: {proposed_total_inventory:,.0f} units")

print("\nBusiness Recommendations:")
print("-" * 50)
if proposed_total_inventory < current_total_inventory and target_service_level > current_service_level:
    print("1. IMPLEMENT the proposed two-stage system:")
    print(f"   - Improves service level by {(target_service_level-current_service_level)*100:.1f}%")
    print(f"   - Reduces total inventory by {current_total_inventory-proposed_total_inventory:,.0f} units")
    print("2. Set up automated reordering based on calculated parameters")
    print("3. Monitor and adjust based on actual demand patterns")
else:
    print("1. REVIEW the current system parameters:")
    print("   - Analyze demand patterns by store location")
    print("   - Consider seasonal variations")
    print("2. Pilot test the new system in selected stores")
    print("3. Develop contingency plans for peak demand periods")